In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

## Felesleges oszlopoktól megszűrt kommentek betöltése

In [ ]:
df1 = pd.read_csv('/kaggle/input/covid2021/covid1.csv', sep = ",", encoding = 'utf8', error_bad_lines=False, quotechar='"')
df2 = pd.read_csv('/kaggle/input/covid2021/covid2.csv', sep = ",", encoding = 'utf8', error_bad_lines=False, quotechar='"')
df3 = pd.read_csv('/kaggle/input/covid2021/covid3.csv', sep = ",", encoding = 'utf8', error_bad_lines=False, quotechar='"')
df4 = pd.read_csv('/kaggle/input/covid2021/covid4.csv', sep = ",", encoding = 'utf8', error_bad_lines=False, quotechar='"')
df5 = pd.read_csv('/kaggle/input/covid2021/covid5.csv', sep = ",", encoding = 'utf8', error_bad_lines=False, quotechar='"')
df6 = pd.read_csv('/kaggle/input/covid2021/covid6.csv', sep = ",", encoding = 'utf8', error_bad_lines=False, quotechar='"')

## Az egészet egyberakjuk

In [ ]:
df12 = df1.append(df2, ignore_index=True)
df123 = df12.append(df3, ignore_index=True)
df1234 = df123.append(df4, ignore_index=True)
df12345 = df1234.append(df5, ignore_index=True)
df_ossz = df12345.append(df6, ignore_index=True)

In [ ]:
df_ossz.shape

2,5 millió komment

## Szótárba rakjuk a kommenteket

In [ ]:
adat = {}
for i in range(len(df_ossz.id)):
    adat[df_ossz.id[i]] = df_ossz["Content of posts"][i]
print(list(adat.items())[0])

## Duplikációk törlése

In [ ]:
rev_multidict = {}
for key, value in adat.items():
    rev_multidict.setdefault(value, set()).add(key)

toremove_all = [list(values) for key, values in rev_multidict.items() if len(values) > 1] # így bekerül a listába az összes ismétlődés, de abból az egyiket szeretnénk megtartani
for i in range(len(toremove_all)):
    del toremove_all[i][-1] #így mindig kiszedünk a duplikációk közül egyet, ami bent marad

In [ ]:
len(toremove_all) #összesen ennyiféle egyező van

In [ ]:
toremove = []
for i in range(len(toremove_all)):
    for j in range(len(toremove_all[i])):
        toremove.append(toremove_all[i][j])
len(toremove) ## ennyi kommentet lehet törölni dublikáció miatt

In [ ]:
df_nodupl = df_ossz[~df_ossz["id"].isin(toremove)]
df_nodupl = df_nodupl.reset_index(drop=True)
df_nodupl.shape

## Kommentek hosszúsága (hány szóból állnak)

In [ ]:
txt_length = []
for i in range(len(df_nodupl["Content of posts"])):
    txt_length.append(len(str(df_nodupl["Content of posts"][i]).split()))

### Komment hosszúságok eloszlása

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

fig, ax = plt.subplots()
sns.distplot(txt_length, ax=ax, hist=True, kde=False, 
             bins=int(100), color = 'blue',
             hist_kws={'edgecolor':'black'})
ax.set_xlim(1,201)
ax.set_xticks(range(1,201, 20))
plt.show()

In [ ]:
fig, ax = plt.subplots()
sns.distplot(txt_length, ax=ax, hist=True, kde=False, 
             bins=range(22), color = 'blue',
             hist_kws={'edgecolor':'black'})
ax.set_xlim(1,21)
ax.set_xticks(range(1,21))
plt.show()

In [ ]:
fig, ax = plt.subplots()
sns.distplot(txt_length, ax=ax, hist=True, kde=False, 
             bins=range(40,202,10), color = 'blue',
             hist_kws={'edgecolor':'black'})
ax.set_ylim(0, 90000)
ax.set_yticks(range(0,90000,10000))
ax.set_xlim(40,201)
ax.set_xticks(range(40,201,10))
plt.show()

## URL-ek törlése

In [ ]:
import re
for i in range(100000):
    df_nodupl["Content of posts"][i] = re.sub("(?P<url>https?://[^\s]+)", "", str(df_nodupl["Content of posts"][i]))

### Ha URL-ek törlése után üres sztring lett a komment, akkor törölje azt a rekordot

In [ ]:
del_elso100000 = df_nodupl[:100000][df_nodupl[:100000]["Content of posts"] == ""]
df_nodup_del1 = df_nodupl[:100000][df_nodupl[:100000]["Content of posts"] != ""]
df_nodup_del1.shape

In [ ]:
import re
for i in range(100000, 300000):
    df_nodupl["Content of posts"][i] = re.sub("(?P<url>https?://[^\s]+)", "", str(df_nodupl["Content of posts"][i]))

In [ ]:
del_más300000 = df_nodupl[100000:300000][df_nodupl[100000:300000]["Content of posts"] == ""]
df_nodup_del2 = df_nodupl[100000:300000][df_nodupl[100000:300000]["Content of posts"] != ""]
df_nodup_del2.shape

In [ ]:
import re
for i in range(300000, 600000):
    df_nodupl["Content of posts"][i] = re.sub("(?P<url>https?://[^\s]+)", "", str(df_nodupl["Content of posts"][i]))

In [ ]:
del_harm600000 = df_nodupl[300000:600000][df_nodupl[300000:600000]["Content of posts"] == ""]
df_nodup_del3 = df_nodupl[300000:600000][df_nodupl[300000:600000]["Content of posts"] != ""]
df_nodup_del3.shape

In [ ]:
import re
for i in range(600000, 900000):
    df_nodupl["Content of posts"][i] = re.sub("(?P<url>https?://[^\s]+)", "", str(df_nodupl["Content of posts"][i]))

In [ ]:
del_harm900000 = df_nodupl[600000:900000][df_nodupl[600000:900000]["Content of posts"] == ""]
df_nodup_del4 = df_nodupl[600000:900000][df_nodupl[600000:900000]["Content of posts"] != ""]
df_nodup_del4.shape

In [ ]:
import re
for i in range(900000, 1200000):
    df_nodupl["Content of posts"][i] = re.sub("(?P<url>https?://[^\s]+)", "", str(df_nodupl["Content of posts"][i]))

In [ ]:
del_negy1200000 = df_nodupl[900000:1200000][df_nodupl[900000:1200000]["Content of posts"] == ""]
df_nodup_del5 = df_nodupl[900000:1200000][df_nodupl[900000:1200000]["Content of posts"] != ""]
df_nodup_del5.shape

In [ ]:
import re
for i in range(1200000, 1500000):
    df_nodupl["Content of posts"][i] = re.sub("(?P<url>https?://[^\s]+)", "", str(df_nodupl["Content of posts"][i]))

In [ ]:
del_ot1500000 = df_nodupl[1200000:1500000][df_nodupl[1200000:1500000]["Content of posts"] == ""]
df_nodup_del6 = df_nodupl[1200000:1500000][df_nodupl[1200000:1500000]["Content of posts"] != ""]
df_nodup_del6.shape

In [ ]:
import re
for i in range(1500000, 1800000):
    df_nodupl["Content of posts"][i] = re.sub("(?P<url>https?://[^\s]+)", "", str(df_nodupl["Content of posts"][i]))

In [ ]:
del_hat1800000 = df_nodupl[1500000:1800000][df_nodupl[1500000:1800000]["Content of posts"] == ""]
df_nodup_del7 = df_nodupl[1500000:1800000][df_nodupl[1500000:1800000]["Content of posts"] != ""]
df_nodup_del7.shape

In [ ]:
import re
for i in range(1800000, 2100000)
    df_nodupl["Content of posts"][i] = re.sub("(?P<url>https?://[^\s]+)", "", str(df_nodupl["Content of posts"][i]))

In [ ]:
del_het2100000 = df_nodupl[1800000:2100000][df_nodupl[1800000:2100000]["Content of posts"] == ""]
df_nodup_del8 = df_nodupl[1800000:2100000][df_nodupl[1800000:2100000]["Content of posts"] != ""]
df_nodup_del8.shape

In [ ]:
import re
for i in range(2100000, len(df_nodupl["Content of posts"])):
    df_nodupl["Content of posts"][i] = re.sub("(?P<url>https?://[^\s]+)", "", str(df_nodupl["Content of posts"][i]))

In [ ]:
del_utolso = df_nodupl[2100000:][df_nodupl[2100000:]["Content of posts"] == ""]
df_nodup_del9 = df_nodupl[2100000:][df_nodupl[2100000:]["Content of posts"] != ""]
df_nodup_del9.shape

In [ ]:
df_nodup_del9.to_csv(r'clean_url9.csv', index = False)